In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppress TensorFlow logging
import warnings
warnings.filterwarnings('ignore')  # Suppress general warnings

from Moduled_functions import get_data
from Moduled_functions import tranformation
from Moduled_functions import calculate_ema_slope
from Moduled_functions import place_order
from m_email import send_email
import pandas as pd
from keras.models import load_model
from sklearn.preprocessing import StandardScaler
from datetime import datetime
import time
from datetime import datetime
import schedule
import numpy as np
import requests

# Suppress specific TensorFlow warnings
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

access_token = os.environ["ACCESS_TOKEN"]
account_id_demo = os.environ["ACCOUNT_DEMO"]
bearish_model = load_model('/work/Trained_Models/FX_Bearish_model_2025-02-19.keras')
bullish_model = load_model('/work/Trained_Models/FX_Bullish_model_2025-02-21.keras')

In [10]:
Bullish_Input = 0.5

In [12]:
Bearish_Input = 0.5

In [18]:
Volume_Input = 300

In [20]:
Move_TopBottom = 0.3

In [3]:
def job():
    try:
        # Get the data using get_data function
        base_data,current_price = get_data(100, access_token)
        
        # Transform the data
        base_data = tranformation(base_data, 0.0015)
        
        # Calculate EMA slopes
        base_data['EMA_slope'] = calculate_ema_slope(base_data, 'Open', 9)
        base_data['EMA_slope_15'] = calculate_ema_slope(base_data, 'Open', 15)
        base_data['EMA_slope_60'] = calculate_ema_slope(base_data, 'Open', 60)
        
        # Calculate deviation
        base_data['deviation_'] = base_data['bullish_move_flag_20'] + base_data['bearish_move_flag_20']
        X = base_data.iloc[:,9:69]
        y = base_data['deviation_']
        
        # Normalize the data
        scaler = StandardScaler()
        scaler.fit(X)
        X_normalized = pd.DataFrame(scaler.transform(X)).round(1)
        
        # Get predictions
        y_pred_bearish = bearish_model.predict(X_normalized, verbose=0)
        y_pred_bearish = pd.DataFrame(y_pred_bearish)
        
        y_pred_bullish = bullish_model.predict(X_normalized, verbose=0)
        y_pred_bullish = pd.DataFrame(y_pred_bullish)
        
        # Combine results
        result_combined = pd.concat([y_pred_bearish, y_pred_bullish, y], axis=1)
        result_combined.columns = ['bearish_0','bearish_1','bearish_2','bearish_3',
                                 'bullish_0','bullish_1','bullish_2','bullish_3','y_true']
        
        base_data = pd.concat([base_data, result_combined], axis=1)

        # Generate flags
        ## bearish indicator
        base_data['bearish_flag']  = base_data.apply(lambda x:2 if x['volume_mean_5'] >450 ##433 
                                                                and x['volume_mean_5'] <750
                                                                and x['bearish_2'] > 0.50 
                                                                and x['bearish_2'] < 0.65 
                                                                #and x['bullish_1'] > 0.509 
                                                                #and x['bullish_1'] < 0.609
                                                                and x['move_from_top'] > 0.08
                                                                and x['move_from_top'] < 0.25
                                                                and x['volume'] > 450 ##410 
                                                                and x['volume'] <700
                                                                and x['volume_mean_3'] > 500
                                                                and x['volume_mean_3'] <750 else 0, axis=1)
        ## bullish indicator
        base_data['bullish_flag'] = base_data.apply(lambda x:1 if x['volume'] >= 450
                                                                and x['volume'] <= 700
                                                                and x['volume_mean_3'] >= 600
                                                                and x['volume_mean_3'] <= 700
                                                                and x['volume_mean_5'] >= 500
                                                                and x['volume_mean_5'] <= 700
                                                                and x['bullish_1'] >= 0.52
                                                                and x['bullish_1'] <= 0.60
                                                                and x['move_from_bottom'] >= 0.12
                                                                and x['move_from_bottom'] <= 0.20 else 0, axis=1)
        
        # Add timestamp column
        current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        base_data['timestamp'] = current_time
        
        # Read existing history
        try:
            existing_data = pd.read_csv('base_data_history.csv')
            existing_data['datetime'] = pd.to_datetime(existing_data['datetime'])
            base_data['datetime'] = pd.to_datetime(base_data['datetime'])
            
            # Fields to update
            update_fields = ['max_high_6', 'min_low_6', 'max_high_20', 'min_low_20',
                           'upward_move_6', 'downward_move_6', 'upward_move_20',
                           'downward_move_20', 'bullish_move_20', 'bearish_move_20',
                           'bullish_move_flag_20', 'bearish_move_flag_20', 'y_true']
            
            # Update existing records
            for datetime_val in base_data['datetime']:
                if datetime_val in existing_data['datetime'].values:
                    # Update fields for matching records
                    mask = existing_data['datetime'] == datetime_val
                    existing_data.loc[mask, update_fields] = base_data[
                        base_data['datetime'] == datetime_val
                    ][update_fields].values
            
            # Add new records
            new_records = base_data[~base_data['datetime'].isin(existing_data['datetime'])]
            if not new_records.empty:
                updated_data = pd.concat([existing_data, new_records])
                updated_data = updated_data.sort_values('datetime')
            else:
                updated_data = existing_data
                
        except FileNotFoundError:
            updated_data = base_data
            
        updated_data.to_csv('base_data_history.csv', index=False)
        
        # Check for trading signals and place orders (rest of the existing code)
        if (base_data['bullish_flag'].iloc[-1] == 1) & (base_data['bearish_flag'].iloc[-1] != 2):
            take_profit = current_price * 1.0015  # 0.15% above
            #stop_loss = current_price * 0.9995    # 0.05% below
            stop_loss = current_price * 0.0005
            qty = '1000000'
            order = place_order(str(current_price),str(round(stop_loss,5)),str(round(take_profit,5)),qty,access_token,account_id_demo)
            order_id = order['orderCreateTransaction'].get('id')
            order_time = order['orderCreateTransaction'].get('time')
    
            message = f"Bullish Order Eligible at this moment. \nOrder placed with order no: {order_id}"
            print(message)
            output =requests.get(f'https://api.day.app/iFbt9PqBdm6d2YvUT4irnN/Bullish Order Placed/{message}')
            #send_email(str(message))

            # Save order details to Excel
            order_data = pd.DataFrame({
                'Time': [order_time],
                'Type': ['BUY'],
                'Quantity': [qty],
                'Price': [current_price],
                'Take_Profit': [take_profit],
                'Stop_Loss': [stop_loss],
                'Order_ID': [order_id]
            })
            
            try:
                existing_orders = pd.read_csv('order_history.csv')
                updated_orders = pd.concat([existing_orders, order_data])
            except FileNotFoundError:
                updated_orders = order_data

            updated_orders.to_csv('order_history.csv', index=False)
            
        elif (base_data['bearish_flag'].iloc[-1] == 2) & (base_data['bullish_flag'].iloc[-1] != 1):
            take_profit = current_price * 0.9985  # 0.15% below
            #stop_loss = current_price * 1.0005    # 0.05% above
            stop_loss = current_price * 0.0005
            qty = '-1000000'
            order = place_order(str(current_price),str(round(stop_loss,5)),str(round(take_profit,5)),qty,access_token,account_id_demo)
            order_id = order['orderCreateTransaction'].get('id')
            order_time = order['orderCreateTransaction'].get('time')
    
            message = f"Bearish Order Eligible at this moment. \nOrder placed with order no: {order_id}"
            print(message)
            output =requests.get(f'https://api.day.app/iFbt9PqBdm6d2YvUT4irnN/Bearish Order Placed/{message}')
            #send_email(str(message))

            # Save order details to Excel
            order_data = pd.DataFrame({
                'Time': [order_time],
                'Type': ['SELL'],
                'Quantity': [qty],
                'Price': [current_price],
                'Take_Profit': [take_profit],
                'Stop_Loss': [stop_loss],
                'Order_ID': [order_id]
            })
            
            try:
                existing_orders = pd.read_csv('order_history.csv')
                updated_orders = pd.concat([existing_orders, order_data])
            except FileNotFoundError:
                updated_orders = order_data

            updated_orders.to_csv('order_history.csv', index=False)

        print(f"Check completed at {current_time}")
        
    except Exception as e:
        print(f"Error occurred: {str(e)}")

In [7]:
# Set up the schedule to run at specified intervals
for minute in [1, 6, 11, 16, 21, 26, 31, 36, 41, 46, 51, 56]:
    schedule.every().hour.at(f":{minute:02d}").do(job)
print("Starting the trading bot...")
while True:
    schedule.run_pending()
    time.sleep(1)

Starting the trading bot...
Check completed at 2025-04-02 22:56:00
Check completed at 2025-04-02 23:01:00
Check completed at 2025-04-02 23:06:00
Check completed at 2025-04-02 23:11:00
Check completed at 2025-04-02 23:16:00
Check completed at 2025-04-02 23:21:00
Check completed at 2025-04-02 23:26:01
Check completed at 2025-04-02 23:31:00
Check completed at 2025-04-02 23:36:01
Check completed at 2025-04-02 23:41:00
Check completed at 2025-04-02 23:46:00
Check completed at 2025-04-02 23:51:01
Check completed at 2025-04-02 23:56:00
Check completed at 2025-04-03 00:01:01
Check completed at 2025-04-03 00:06:00
Check completed at 2025-04-03 00:11:00
Check completed at 2025-04-03 00:16:00
Check completed at 2025-04-03 00:21:00
Check completed at 2025-04-03 00:26:00
Check completed at 2025-04-03 00:31:01
Check completed at 2025-04-03 00:36:00
Check completed at 2025-04-03 00:41:00
Check completed at 2025-04-03 00:46:01
Check completed at 2025-04-03 00:51:00
Check completed at 2025-04-03 00:56:

KeyboardInterrupt: 

# Future set up blocks

In [ ]:
base_data['bullish_flag'] = base_data.apply(lambda x: 1 if x['bullish_1']>0.6
                                        and x['bearish_2'] < 0.6
                                        and x['volume'] > 300 
                                        and x['volume'] < 800
                                        and x['EMA_slope']>1 
                                        and x['EMA_slope']<4 
                                        and x['EMA_slope_60']>0
                                        and x['move_from_bottom'] < 0.26
                                        else 0, axis=1)

base_data['bearish_flag'] = base_data.apply(lambda x: 2 if x['bearish_2']>0.6 
                                        and x['bullish_1'] < 0.6
                                        and x['volume'] > 300
                                        and x['volume'] < 800
                                        and x['bearish_2'] >  x['bullish_1']
                                        and x['EMA_slope'] < -1
                                        and x['EMA_slope']>-4 
                                        and x['EMA_slope_60']<0
                                        and x['move_from_top'] < 0.26
                                        else 0, axis=1)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a127aa9e-0a77-4af9-a6ce-85e7a9b74042' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>